<p>  The dataset is available on Kaggle.
link: https://www.kaggle.com/c/sentiment-analysis-on-movie-reviews/data   </p>

In [1]:
import pandas as pd

train_data = 'F:/Shreya_Internship/73 strings/train.tsv/train.tsv'
test_data = 'F:/Shreya_Internship/73 strings/test.tsv/test.tsv'
url_train = pd.read_table(train_data , sep = '\t' , engine='python' )
url_test = pd.read_table(test_data , sep = '\t' , engine='python' )

url_train.head()

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,A series of escapades demonstrating the adage ...,1
1,2,1,A series of escapades demonstrating the adage ...,2
2,3,1,A series,2
3,4,1,A,2
4,5,1,series,2


In [2]:
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import re
from nltk.corpus import stopwords

In [3]:
def clean(sent): 
    new_sent = sent['Phrase'] 
    new_sent = re.sub('[^a-zA-Z]', ' ',new_sent)
    new_sent = new_sent.lower()
    new_sent = word_tokenize(new_sent)
    lemmatizer = WordNetLemmatizer()
    new_sent = [lemmatizer.lemmatize(w) for w in new_sent if not w in set(stopwords.words('english'))]
    return (' '.join(new_sent))

In [4]:
url_train['new_Phrase'] = url_train.apply(clean , axis=1)
url_test['new_Phrase'] = url_test.apply(clean , axis=1)


print(url_train.head())

   PhraseId  SentenceId                                             Phrase  \
0         1           1  A series of escapades demonstrating the adage ...   
1         2           1  A series of escapades demonstrating the adage ...   
2         3           1                                           A series   
3         4           1                                                  A   
4         5           1                                             series   

   Sentiment                                         new_Phrase  
0          1  series escapade demonstrating adage good goose...  
1          2     series escapade demonstrating adage good goose  
2          2                                             series  
3          2                                                     
4          2                                             series  


In [7]:
from keras.preprocessing.text import Tokenizer
token=Tokenizer()
token.fit_on_texts(url_train['new_Phrase'])

In [8]:
X_train=token.texts_to_sequences(url_train['new_Phrase'])
print(X_train)

[[230, 6296, 5517, 6013, 11, 2725, 103, 11, 9797, 553, 9111, 479, 554, 15, 7], [230, 6296, 5517, 6013, 11, 2725], [230], [], [230], [6296, 5517, 6013, 11, 2725], [], [6296, 5517, 6013, 11, 2725], [6296], [5517, 6013, 11, 2725], [5517, 6013], [5517], [6013], [], [6013], [11, 2725], [], [11, 2725], [], [11, 2725], [], [11, 2725], [11], [2725], [], [2725], [2725], [103, 11, 9797, 553, 9111, 479, 554, 15, 7], [103, 11, 9797, 553, 9111, 479, 554, 15, 7], [103], [103], [11, 9797, 553, 9111, 479, 554, 15, 7], [9797, 553, 9111, 479, 554, 15, 7], [9797, 553, 9111, 479, 554, 15, 7], [9797], [9797], [9797], [], [553, 9111, 479, 554, 15, 7], [], [], [], [], [553, 9111, 479, 554, 15, 7], [553], [9111, 479, 554, 15, 7], [9111], [479, 554, 15, 7], [], [479, 554, 15, 7], [479], [554, 15, 7], [554, 15, 7], [554, 15, 7], [554], [15, 7], [], [15, 7], [15], [7], [7], [7], [], [697, 4726, 121, 2791, 154, 2214], [697, 4726, 121, 2791], [], [697, 4726, 121, 2791], [697, 4726, 121], [697], [4726, 121], [4726,

In [9]:
def mtoken(count):
    s=count['new_Phrase'].split(' ')
    return len(s)

url_train['len']=url_train.apply(mtoken,axis=1)
url_train.head()

,PhraseId,SentenceId,Phrase,Sentiment,new_Phrase,len
0,1,1,A series of escapades demonstrating the adage ...,1,series escapade demonstrating adage good goose...,15
1,2,1,A series of escapades demonstrating the adage ...,2,series escapade demonstrating adage good goose,6
2,3,1,A series,2,series,1
3,4,1,A,2,,1
4,5,1,series,2,series,1


In [10]:
max(url_train['len'])

30

In [11]:
url_test['len']=url_test.apply(mtoken,axis=1)
max(url_test['len'])

27

In [12]:
from keras.preprocessing.sequence import pad_sequences
X_train=pad_sequences(X_train,maxlen=30,padding='post')
print(X_train)

[[ 230 6296 5517 ...    0    0    0]
 [ 230 6296 5517 ...    0    0    0]
 [ 230    0    0 ...    0    0    0]
 ...
 [8467 8468    0 ...    0    0    0]
 [8467    0    0 ...    0    0    0]
 [8468    0    0 ...    0    0    0]]


In [13]:
from sklearn.preprocessing import OneHotEncoder

ohe=OneHotEncoder()

Y_train=url_train.iloc[:,3]
Y_train=Y_train.as_matrix()
Y_train=Y_train.reshape(-1,1)
Y_tr=ohe.fit_transform(Y_train)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


In [14]:
from keras.models import Sequential
from keras.layers import Dense,Dropout,Embedding,LSTM

len(token.word_index)

13625

In [19]:
model=Sequential()
model.add(Embedding(13626,300))
model.add(LSTM(150))
model.add(Dense(5,activation='softmax'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, None, 300)         4087800   
_________________________________________________________________
lstm_2 (LSTM)                (None, 150)               270600    
_________________________________________________________________
dense_2 (Dense)              (None, 5)                 755       
Total params: 4,359,155
Trainable params: 4,359,155
Non-trainable params: 0
_________________________________________________________________


In [20]:
model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [21]:
model.fit(X_train,Y_tr,batch_size=64,epochs=15)

Epoch 1/15
156060/156060 [==============================] - 789s 5ms/step - loss: 1.0160 - acc: 0.5956
Epoch 2/15
156060/156060 [==============================] - 659s 4ms/step - loss: 0.8098 - acc: 0.6702
Epoch 3/15
156060/156060 [==============================] - 647s 4ms/step - loss: 0.7390 - acc: 0.6975
Epoch 4/15
156060/156060 [==============================] - 640s 4ms/step - loss: 0.6910 - acc: 0.7144
Epoch 5/15
156060/156060 [==============================] - 653s 4ms/step - loss: 0.6543 - acc: 0.7266
Epoch 6/15
156060/156060 [==============================] - 636s 4ms/step - loss: 0.6227 - acc: 0.7365
Epoch 7/15
156060/156060 [==============================] - 783s 5ms/step - loss: 0.5954 - acc: 0.7451
Epoch 8/15
156060/156060 [==============================] - 776s 5ms/step - loss: 0.5703 - acc: 0.7536
Epoch 9/15
156060/156060 [==============================] - 871s 6ms/step - loss: 0.5472 - acc: 0.7607
Epoch 10/15
156060/156060 [==============================] - 787s 5ms/ste

In [22]:
X_test=token.texts_to_sequences(url_test['new_Phrase'])
X_test=pad_sequences(X_test,maxlen=56,padding='post')

X_test.shape

(66292, 56)

In [23]:
import numpy as np

s=model.predict(X_test)
s=np.argmax(s,axis=1)
print(s)
s=pd.DataFrame(s)
s['PhraseId']=url_test['PhraseId']
s.columns=['Sentiment','PhraseId']
s=s[['PhraseId','Sentiment']]
s.to_csv('F:/Shreya_Internship/73 strings/results.csv',index=False)

[3 3 2 ... 1 1 2]


In [24]:
print(s)

       PhraseId  Sentiment
0        156061          3
1        156062          3
2        156063          2
3        156064          3
4        156065          3
5        156066          3
6        156067          3
7        156068          2
8        156069          3
9        156070          2
10       156071          2
11       156072          2
12       156073          2
13       156074          2
14       156075          2
15       156076          3
16       156077          3
17       156078          3
18       156079          3
19       156080          2
20       156081          2
21       156082          2
22       156083          2
23       156084          2
24       156085          2
25       156086          2
26       156087          2
27       156088          2
28       156089          3
29       156090          2
...         ...        ...
66262    222323          3
66263    222324          2
66264    222325          1
66265    222326          2
66266    222327          3
6